In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 46.9 MB 44 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 12.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=6113624421f5c68e31fd0b7919bb4cbaeab108c5b4444ba4f4b47cccd64d8a5d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 11.0 MB/s 
     |████████████████████████████████| 2.8 MB 11.9 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
     |████████████████████████████████| 636 kB 39.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 25.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.1

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ds1c2nb3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ds1c2nb3
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=5b339cc476a6b2961d436fcbd1eaeff07c8d02e161fad53fae1b932c1a0195cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-bj5or_u0/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [13]:
import pandas as pd

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [8]:
!wget https://drive.google.com/file/d/1tSUWawR4pzyOD3DhQ-hWhLpc0QrdViuV/view?usp=sharing -O mac_train.csv

--2021-09-14 06:26:23--  https://drive.google.com/file/d/1tSUWawR4pzyOD3DhQ-hWhLpc0QrdViuV/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.251.5.101, 142.251.5.102, 142.251.5.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.5.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘mac_train.csv’

mac_train.csv           [ <=>                ]  64.50K  --.-KB/s    in 0.07s   

2021-09-14 06:26:23 (913 KB/s) - ‘mac_train.csv’ saved [66047]



In [11]:
!ls -l

total 72
-rw-r--r-- 1 root root 66047 Sep 14 06:26 mac_train.csv
drwxr-xr-x 1 root root  4096 Sep  1 19:26 sample_data


In [16]:
train_data = pd.read_csv('./drive/MyDrive/mac_train.csv')

In [17]:
train_data.head()

,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL
0,올리즈원,@olliez_one,2021-09-01T15:24:46.000Z,올리즈원\n@olliez_one\n·\nSep 1,강혜원 맥도날드 반대운동\n강혜원 절대 죽먹어\n\n계란죽 끓이는법(초간단)\n1....,🥕,25.0,134.0,224.0,['https://pbs.twimg.com/media/E-NaWYwVQAQ3L3J?...,https://twitter.com/olliez_one/status/14330884...
1,마늘장아찌,@drunken9arlic,2021-09-01T10:02:02.000Z,마늘장아찌\n@drunken9arlic\n·\nSep 1,121111(장소 : 맥도날드)\n1\n1,NaN,1.0,1.0,NaN,['https://pbs.twimg.com/media/E-MQebeUUAMekvs?...,https://twitter.com/drunken9arlic/status/14330...
2,not_Joon,@JoonNot,2021-09-01T12:19:49.000Z,not_Joon\n@JoonNot\n·\nSep 1,자유공원 맥아더씨 동상. \n\n그가 바라보고 있는 시선을 기준으로 180° 방향으...,NaN,3.0,NaN,5.0,['https://pbs.twimg.com/media/E-MwAFhVgAAX7m0?...,https://twitter.com/JoonNot/status/14330419036...
3,빡정(동물농장),@XWdwcIn5XihkEpX,2021-09-01T12:16:35.000Z,빡정(동물농장)\n@XWdwcIn5XihkEpX\n·\nSep 1,요즘 이게 없으면 안된다!하는거는?이란 질문에 다들 농담삼아 햄버거~ㅋㅋ맥도날드~ㅋ...,NaN,NaN,NaN,4.0,['https://pbs.twimg.com/media/E-MvRzCVUAkc8Ns?...,https://twitter.com/XWdwcIn5XihkEpX/status/143...
4,정치하는엄마들 공식계정,@MamasPolitical,2021-09-01T04:29:30.000Z,정치하는엄마들 공식계정\n@MamasPolitical\n·\nAug 31,[연대/맥도날드에게 사회적 책임을 촉구하는 대책위]\n\n위생불량 맥도날드!\n ...,💥,1.0,2.0,NaN,['https://pbs.twimg.com/media/E-LEXdnUYAEinOi?...,https://twitter.com/MamasPolitical/status/1432...


In [19]:
train_dataset = []
for sen, label in zip(train_data['UserName'], train_data['Embedded_text']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

In [21]:
print(train_dataset)

[['@olliez_one', '강혜원 맥도날드 반대운동\n강혜원 절대 죽먹어\n\n계란죽 끓이는법(초간단)\n1.냄비에 밥이랑 물을 넣고 끓인다\n   (물은 밥 양과 같은 높이정도)\n2.보글보글하면 계란을 2개 넣음\n3.휙휙 저어주면서 간장으로 간맞춤\n4.마무리로 파같은거 넣기(개취)\n5.소금후추로 간맞춤 (개취)\n#혜원메일 #혜원버블\n25\n134\n224'], ['@drunken9arlic', '121111(장소 : 맥도날드)\n1\n1'], ['@JoonNot', '자유공원 맥아더씨 동상. \n\n그가 바라보고 있는 시선을 기준으로 180° 방향으로 쭉 가면 놀랍게도 맥도날드가 나옴.\n3\n5'], ['@XWdwcIn5XihkEpX', '요즘 이게 없으면 안된다!하는거는?이란 질문에 다들 농담삼아 햄버거~ㅋㅋ맥도날드~ㅋㅋ이랬는데 됴0이가 팬분들!이라고해서 농담하던 사람들 다 주먹울음중\n4'], ['@MamasPolitical', '[연대/맥도날드에게 사회적 책임을 촉구하는 대책위]\n\n위생불량 맥도날드!\n      나는 불매한다!\n\n#정치하는엄마들 은 온라인과 오프라인으로 1인 시위 함께 하고 있습니다. \n\n#맥도날드 #맥도날드OUT #맥도날드불매운동 #식품안전 #부당징계철회 #모두가엄마다 #정치하는엄마들\n1\n2'], ['@MamasPolitical', '[1인시위] 위생불량 맥도날드, 나는 불매한다!\n\n#맥도날드 #맥도날드OUT #맥도날드불매운동 #식품안전 #부당징계철회 #모두가엄마다 #정치하는엄마들\n\n인증사진을 위 #해쉬태그 달아 SNS에 공유해주시거나 act@politicalmamas.kr 로 보내주시면 웹자보에 추가해드립니다. \n\n함께 해주세요!'], ['@Apex_ikki', '뭔 맥도날드가 가 드라이브 스루점밖에 업냐\n2\n4'], ['@manman_E2', '맥도날드 귀엽게 먹는거 무슨일이냐고\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\nㅠ-ㅠ)~~\nThe following media includes p

In [22]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [23]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [25]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [29]:
#data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

ValueError: ignored